# 3.1 Teilen Sie Ihren Datensatz in einen test- und einen train-Satz ein.

Teil 1: Import der Daten:

In [72]:
%pip install -U scikit-learn

import pandas as pd
import os

bitcoin = os.path.join("Datenset_LB_M259 - Demo.csv")
Bitcoin = pd.read_csv(bitcoin)
Bitcoin.head()

Note: you may need to restart the kernel to use updated packages.


,time,low,high,open,close,volume
0,2021-10-30 03:00:00,61868.81,61920.00,61888.18,61919.98,2.946079
1,2021-10-30 02:59:00,61882.74,61912.67,61903.16,61882.74,2.942357
2,2021-10-30 02:58:00,61854.82,61907.96,61854.82,61903.15,3.047848
3,2021-10-30 02:57:00,61848.04,61883.02,61883.01,61851.39,3.381070
4,2021-10-30 02:56:00,61879.33,61925.42,61907.47,61882.28,5.663128


Teil 2: Bestimmen von *features* und *targets*

In [73]:
# x = features
X = Bitcoin[["low", "open", "close", "volume"]]

# y = targets
y = Bitcoin[["high"]]
X.head()

,low,open,close,volume
0,61868.81,61888.18,61919.98,2.946079
1,61882.74,61903.16,61882.74,2.942357
2,61854.82,61854.82,61903.15,3.047848
3,61848.04,61883.01,61851.39,3.381070
4,61879.33,61907.47,61882.28,5.663128


Teil 3: *preprocessing*

In [74]:

y=y.astype("int")
# Dieser Codeabschnitt gibt die Anzahl zeilen zurück die NaN enthalten.
len(X[X.isna().any(axis=1)])
from sklearn.impute import SimpleImputer
import numpy as np
si = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
si.fit(X)
X_ = si.transform(X)
X = pd.DataFrame(X_, columns=X.columns)
len(X[X.isna().any(axis=1)])
# Da die Ausgabe gleich Null ist, muss ich nichts weiteres unternehmen.
# Ausserdem habe ich keine NaN Felder. Deshalb muss ich auch da nichts unternehmen.

0

Teil 4: *test-train-split*

In [75]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size = 0.2, random_state = 42
)
len(X_train), len(X_test)

(799, 200)

Teil 5: Algorithmuswahl

In [76]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

algorithms = {
    # "Nearest Neighbors" : KNeighborsClassifier(3),
    # "Stochastic Gradient Descent" : SGDClassifier(),
    # "Linear SVM" : SVC(kernel="linear", C=0.025),
    # "Decision Tree" : DecisionTreeClassifier(max_depth=5),
    # "Random Forest" : RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    # "Neural Net" : MLPClassifier(alpha=1, max_iter=1000),
    "Naive Bayes" : GaussianNB(),
    # "LDA" : LinearDiscriminantAnalysis(),
}


for name, algorithm in algorithms.items():
    algorithm.fit(X_train, y_train.values.ravel())

    score = algorithm.score(X_test, y_test)
    print(name, round(score,2))

Naive Bayes 0.04


Nearest Neighbors 0.2

Stochastic Gradient Descent | kein Ergebnis

Linear SVM | kein Ergebnis

Decision Tree 0.21

Random Forest 0.21

Neural Net 0.2

Naive Bayes 0.2

LDA 0.2

# 3.2 Algorithmus bestimmen
Aufgrund der Ergebnisse durch das testen der verschiedenen sklearn Algorithmen, bin ich auf den Algorithmus mit dem Namen Decision Tree gekommen. Diese war der Algorithmus mit der grössten Genauigkeit.

# 3.3 Testen

In [77]:
#Medell speichern
import joblib
joblib.dump(algorithms["Naive Bayes"], 'bitcoin_Naive_Bayes.joblib')
best_model = joblib.load('bitcoin_Naive_Bayes.joblib')
best_model

GaussianNB()

In [78]:
leo = pd.DataFrame([[61868.81, 61888.18, 61919.98, 2.94607889]], columns=X_train.columns)
leo.head()

Sahra = pd.DataFrame([[61882.74, 61903.16, 61882.74, 2.94235679]], columns=X_train.columns)
Sahra.head()

Michelle = pd.DataFrame([[61854.82, 61854.82, 61903.15, 3.0478475]], columns=X_train.columns)
Michelle.head()


,low,open,close,volume
0,61854.82,61854.82,61903.15,3.047848


In [80]:
predictions1 = best_model.predict(pd.concat([leo]))
for person, prediction1 in zip(["Leo"], predictions1):
    print("Erwartet: 61920, Berechnet:", prediction1)

predictions2 = best_model.predict(pd.concat([Sahra]))
for person, prediction2 in zip(["Sahra"], predictions2):
    print("Erwartet: 61912.67, Berechnet:", prediction2)

predictions3 = best_model.predict(pd.concat([Michelle]))
for person, prediction3 in zip(["Michelle"], predictions3):
    print("Erwartet: 61907.96, Berechnet:", prediction3)

Erwartet: 61920, Berechnet: 61920
Erwartet: 61912.67, Berechnet: 61912
Erwartet: 61907.96, Berechnet: 61907


Test Erkenntnisse:
Nach meinen Algorithmus-Tests nach, sollten die Ergebnisse 4% Genauigkeit aufweisen. Dies ist nicht sehr viel, daher habe ich auch nicht viel erwartet. Die Test aber zeigten eine Genauigkeit von 100% auf, was umso besser ist. Die Ergebnisse sind dabei auf ganze Zahlen gerunden.